<a href="https://colab.research.google.com/github/Deividasbu/Data-Science-projects/blob/main/2.3.4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import psycopg2

db_connection = psycopg2.connect(
    database="dfsajvobi21n3d",
    user="ywpumyzacvunhd",
    password="876f5866e83a383112e351f683992b310c46e8a07bc5ebb8df56ed48728bf44a",
    host="ec2-54-155-87-214.eu-west-1.compute.amazonaws.com",
    port="5432"
)

cur = db_connection.cursor()

cur.execute("""
DROP TABLE IF EXISTS cars CASCADE;
"""
)

db_connection.commit()

cur.execute('''
CREATE TABLE cars (
    id serial PRIMARY KEY,
    model varchar(255),
    color varchar(255)
);
''')

db_connection.commit()
db_connection.close()

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
import threading
import json

from flask import Flask, request
from werkzeug import serving
import psycopg2

db_connection = psycopg2.connect(
    database="dfsajvobi21n3d",
    user="ywpumyzacvunhd",
    password="876f5866e83a383112e351f683992b310c46e8a07bc5ebb8df56ed48728bf44a",
    host="ec2-54-155-87-214.eu-west-1.compute.amazonaws.com",
    port="5432"
)

app = Flask(__name__)

@app.route("/cars", methods=["GET", "POST"])
def cars() -> str:
    if request.method == "GET":
        cur = db_connection.cursor()
        try:
            # COLLECTING LIST OF ALL CARS
            cur.execute("SELECT * FROM cars")
            rows = cur.fetchall()
            cars = [{"id": row[0], "model": row[1], "color": row[2]} for row in rows]
            return json.dumps({"cars": cars})
        except:
            return json.dumps({"error": "CANNOT RETRIEVE CARS"}), 500
    
    if request.method == "POST":
        cur = db_connection.cursor()
        try:
            post_data = json.loads(request.data)
            cur.execute(f"INSERT INTO cars(model, color) VALUES('{post_data['car']['model']}', '{post_data['car']['color']}')")
            db_connection.commit()
            #
            cur.execute("SELECT * FROM cars")
            rows = cur.fetchall()
            return json.dumps({"car": {"id": rows[-1][0], "model": rows[-1][1], "color": rows[-1][2]}})
        except:
            return json.dumps({"error": "CANNOT RETRIEVE USERS"}), 500 

@app.route("/cars/<car_id>", methods=["GET", "PUT", "DELETE"])
def car(car_id: str) -> str: 
      if request.method == "GET":
        cur = db_connection.cursor()
        try:
            # 
            cur.execute(f"SELECT cars.id, cars.model, cars.color FROM cars WHERE cars.id={car_id}")
            rows = cur.fetchall()
            cars = [{"id": row[0], "model": row[1], "color": row[2]} for row in rows]
            return json.dumps({"car": cars})
        except:
            return json.dumps({"error": "CANNOT RETRIEVE CARS"}), 500

      if request.method == "PUT":
        cur = db_connection.cursor()
        try:
            post_data = json.loads(request.data)
            cur.execute(f"UPDATE cars SET model='{post_data['car']['model']}', color='{post_data['car']['color']}' WHERE id={car_id}")
            print(f"UPDATE cars SET model='{post_data['car']['model']}', color='{post_data['car']['color']}' WHERE id={car_id}")
            db_connection.commit()
            rows = cur.fetchall()
            cars = [{"id": row[0], "model": row[1], "color": row[2]} for row in rows]
            return json.dumps({"car": cars})
        except:
            return json.dumps({"error": "CANNOT RETRIEVE CARS"}), 500

      if request.method == 'DELETE':
        cur = db_connection.cursor()
        try: 
            cur.execute(f"DELETE FROM cars WHERE id = {car_id}")
            db_connection.commit()
            return json.dumps({"SUCCESS": "CAR WAS DELETED"}), 200
        except:
            return json.dumps({"error": "CANNOT DELETE CAR BY THIS ID"}), 500



t = threading.Thread(target=serving.run_simple, args=('localhost', 9000, app))
t.start()


 * Running on http://localhost:9000/ (Press CTRL+C to quit)


In [7]:
resp = requests.put("http://localhost:9000/cars/1", data=json.dumps({"car": {"model": "Tesla Y", "color": "black"}}))
print(json.loads(resp.text))


127.0.0.1 - - [25/Mar/2021 08:09:11] "PUT /cars/1 HTTP/1.1" 500 -


UPDATE cars SET model='Tesla Y', color='black' WHERE id=1
{'error': 'CANNOT RETRIEVE CARS'}


In [6]:
import requests
import json
resp = requests.post("http://localhost:9000/cars", data=json.dumps({"car": {"model": "Tesla S", "color": "black"}}))
print(json.loads(resp.text))

resp = requests.put("http://localhost:9000/cars/1", data=json.dumps({"car": {"model": "Tesla Y", "color": "black"}}))
print(json.loads(resp.text))

resp = requests.delete("http://localhost:9000/cars/1")
print(json.loads(resp.text))

resp = requests.get("http://localhost:9000/cars/1")
print(json.loads(resp.text))

127.0.0.1 - - [25/Mar/2021 07:56:11] "POST /cars HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 07:56:11] "PUT /cars/1 HTTP/1.1" 500 -


{'car': {'id': 4, 'model': 'Tesla S', 'color': 'black'}}
UPDATE cars SET model='Tesla Y', color='black' WHERE id=1
{'error': 'CANNOT RETRIEVE CARS'}


127.0.0.1 - - [25/Mar/2021 07:56:12] "DELETE /cars/1 HTTP/1.1" 200 -
127.0.0.1 - - [25/Mar/2021 07:56:12] "GET /cars/1 HTTP/1.1" 200 -


{'SUCCESS': 'CAR WAS DELETED'}
{'car': []}


In [ ]:
# RESTARTING KERNEL TO STOP THE BACKGROUND THREAD
import os
os._exit(00)